# New Section

In [17]:
%matplotlib
import matplotlib.pyplot as plt

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import datetime

import joblib
import pickle
import tempfile


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


Using matplotlib backend: agg


In [18]:
model_data='s3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-12-11-28-10-280/output/model.tar.gz'

In [19]:
model_data

's3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-12-11-28-10-280/output/model.tar.gz'

In [20]:
from sagemaker import image_uris


In [21]:
image=image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

In [22]:
image 

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [23]:
role

'arn:aws:iam::479862333671:role/service-role/AmazonSageMaker-ExecutionRole-20220103T082598'

In [24]:
trainedmodel = sagemaker.model.Model(
    image_uri=image,
    model_data = model_data,
    role=role)

In [25]:
model_transformer = trainedmodel.transformer(instance_count=1, instance_type='ml.c5.4xlarge')

In [28]:
test = pd.read_csv('s3://{}/{}'.format(bucket, 'data/test.csv'))
test

,1437,3,1,0.1,0.2,1914,30,4,4.1,2,0.3,0.4,0.5,8.38,25,17,4.2,5,4.3,0.0,0.0.1,0.0.2,1.0,0.0.3,0.0.4,0.0.5,0.0.6,0.0.7,0.0.8
0,1437,3,1,0,0,1915,30,4,4,2,0,0,0,8.38,26,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1437,3,1,0,0,1916,30,4,4,2,0,0,0,8.38,27,17,4,5,4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1437,3,1,0,0,1917,30,4,4,2,0,0,0,8.38,28,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1437,3,1,0,0,1918,30,4,4,2,0,0,0,8.38,29,17,4,5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1438,3,1,0,0,1914,30,4,4,2,0,0,0,3.97,25,17,4,5,4,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823224,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
823225,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
823226,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
823227,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [31]:
val = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))
val

,0,1437,3,1,0.1,0.2,1914,30,4,4.1,2,0.3,0.4,0.5,8.38,25,17,4.2,5,4.3,0.0,0.0.1,0.0.2,1.0,0.0.3,0.0.4,0.0.5,0.0.6,0.0.7,0.0.8
0,0,1437,3,1,0,0,1915,30,4,4,2,0,0,0,8.38,26,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0,1437,3,1,0,0,1916,30,4,4,2,0,0,0,8.38,27,17,4,5,4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,1437,3,1,0,0,1917,30,4,4,2,0,0,0,8.38,28,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,1437,3,1,0,0,1918,30,4,4,2,0,0,0,8.38,29,17,4,5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1438,3,1,0,0,1914,30,4,4,2,0,0,0,3.97,25,17,4,5,4,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823224,1,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
823225,1,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
823226,0,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
823227,1,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [32]:
val_location = 's3://{}/{}'.format(bucket, 'data/val.csv')

In [36]:
xgb_transformer = trainedmodel.transformer(instance_count=1, instance_type='ml.c5.4xlarge')

In [ ]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

In [34]:
model_transformer.transform(val_location, content_type='text/csv', split_type='Line')

...............................[2022-01-21:16:54:46:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-21:16:54:46:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-21:16:54:46:INFO] nginx config: 
worker_processes auto;
daemon off;
[2022-01-21:16:54:46:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-21:16:54:46:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-21:16:54:46:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_

UnexpectedStatusException: Error for Transform job sagemaker-xgboost-2022-01-21-16-49-48-489: Failed. Reason: ClientError: See job logs for more information

In [ ]:
model_transformer.wait()

In [ ]:
!aws s3 cp --recursive $model_transformer.output_path $data_dir

In [ ]:
Y_pred = pd.read_csv('s3://{}/{}'.format(bucket, 'data/X_val_values.csv.out'), header=None)
Y_pred.columns = ['demand']

In [ ]:
Y_pred

In [ ]:
Y_test = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))['demand'].to_frame()

In [ ]:
Y_test

In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mean_squared_error(Y_test, Y_pred, squared=True)


6.131172588601535

In [24]:
mean_squared_error(Y_test, Y_pred, squared=False)

2.476120471342526

In [26]:
mean_absolute_error(Y_test, Y_pred)

1.2007927503676232